In [1]:
from google.colab import drive
import os
import cv2
import math
import numpy as np
from skimage.metrics import structural_similarity as ssim
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
# Defining Image Quality Metric Functions
def psnr(target, ref):
    target_data = target.astype(float)
    ref_data = ref.astype(float)
    diff = ref_data - target_data
    diff = diff.flatten('C')
    rmse = math.sqrt(np.mean(diff ** 2.))
    return 20 * math.log10(255. / rmse)

def mse(target, ref):
    err = np.sum((target.astype('float') - ref.astype('float')) ** 2)
    err /= float(target.shape[0] * target.shape[1])
    return err

def compare_images(target, ref):
    return [psnr(target, ref), mse(target, ref), ssim(target, ref, multichannel=True)]

# Defining the SRCNN Model
def model():
    SRCNN = Sequential([
        Conv2D(128, (9, 9), activation='relu', padding='same', input_shape=(None, None, 1), kernel_initializer='glorot_uniform'),
        Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='glorot_uniform'),
        Conv2D(1, (5, 5), activation='linear', padding='same', kernel_initializer='glorot_uniform')
    ])
    SRCNN.compile(optimizer=Adam(lr=0.0003), loss='mean_squared_error', metrics=['mean_squared_error'])
    return SRCNN

# Function to prepare images
def prepare_images(path, factor, save_dir):
    try:
        files = os.listdir(path)
        print(f"Found {len(files)} files in {path}")
    except OSError as e:
        print(f"Error reading directory {path}: {e}")
        return

    for file in files:
        try:
            img_path = os.path.join(path, file)
            img = cv2.imread(img_path)
            h, w, _ = img.shape
            img = cv2.resize(img, (w // factor, h // factor), interpolation=cv2.INTER_LINEAR)
            img = cv2.resize(img, (w, h), interpolation=cv2.INTER_LINEAR)
            cv2.imwrite(os.path.join(save_dir, file), img)
            print(f"Processed and saved: {file}")
        except Exception as e:
            print(f"Error processing file {file}: {e}")


In [4]:
# Prepare directories
source_dir = '/content/drive/MyDrive/aqua_data/non_invasive'
degraded_dir = '/content/drive/MyDrive/SRCNN/degrade_non'
output_dir = '/content/drive/MyDrive/SRCNN/output_non'






In [5]:
# Run the preparation function
prepare_images(source_dir, 2, degraded_dir)

Streaming output truncated to the last 5000 lines.
Processed and saved: Image0343_027.png
Processed and saved: Image0444_032.png
Processed and saved: Image0453_018.png
Processed and saved: Image0449_024.png
Processed and saved: Image0446_029.png
Processed and saved: Image0451_024.png
Processed and saved: Image0324_060.png
Processed and saved: Image0445_031.png
Processed and saved: Image0445_032.png
Processed and saved: Image0450_024.png
Processed and saved: Image0346_025.png
Processed and saved: Image0443_035.png
Processed and saved: Image0451_023.png
Processed and saved: Image0452_021.png
Processed and saved: Image0337_041.png
Processed and saved: Image0449_025.png
Processed and saved: Image0321_065.png
Processed and saved: Image0345_028.png
Processed and saved: Image0458_010.png
Processed and saved: Image0460_009.png
Processed and saved: Image0344_027.png
Processed and saved: Image0436_038.png
Processed and saved: Image0442_034.png
Processed and saved: Image0459_010.png
Processed and

In [6]:
# Load the SRCNN model
srcnn = model()

# Load weights if you have them (optional)
srcnn.load_weights('/content/drive/MyDrive/SRCNN/3051crop_weight_200.h5')



In [ ]:
# Function to predict and enhance images
def predict_and_enhance(image_path, srcnn):
    path, file = os.path.split(image_path)
    degraded = cv2.imread(image_path)
    ref = cv2.imread(os.path.join(source_dir, file))

    # Convert to grayscale for SRCNN
    temp = cv2.cvtColor(degraded, cv2.COLOR_BGR2YCrCb)
    Y = np.zeros((1, temp.shape[0], temp.shape[1], 1), dtype=float)
    Y[0, :, :, 0] = temp[:, :, 0].astype(float) / 255

    # Predict with SRCNN
    pre = srcnn.predict(Y, batch_size=1)
    pre = pre * 255
    pre[pre > 255] = 255
    pre[pre < 0] = 0
    pre = pre.astype(np.uint8)

    # Restore color and save
    temp[:, :, 0] = pre[0, :, :, 0]
    output = cv2.cvtColor(temp, cv2.COLOR_YCrCb2BGR)
    cv2.imwrite(os.path.join(output_dir, file), output)

    # Calculate image quality scores
    scores = compare_images(output, ref)
    print(f'Processed {file} with PSNR: {scores[0]}, MSE: {scores[1]}, SSIM: {scores[2]}')

# Process each degraded image
for file in os.listdir(degraded_dir):
    try:
        predict_and_enhance(os.path.join(degraded_dir, file), srcnn)
    except Exception as e:
        print(f'Failed to process {file}: {e}')


1/1 [==============================] - 0s 499ms/step
Processed Imageio560_025.png with PSNR: 28.4018829340892, MSE: 281.8469387755102, SSIM: 0.9804731480366954
1/1 [==============================] - 0s 93ms/step


<ipython-input-2-8f5848f74ffb>:16: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  return [psnr(target, ref), mse(target, ref), ssim(target, ref, multichannel=True)]


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 39ms/step
Processed Image0319_046.png with PSNR: 31.018423910309618, MSE: 154.29761904761904, SSIM: 0.9766221499468983
1/1 [==============================] - 0s 42ms/step
Processed Image0321_044.png with PSNR: 32.29381695873788, MSE: 115.03233082706767, SSIM: 0.9552067289715286
1/1 [==============================] - 0s 35ms/step
Processed Image0337_018.png with PSNR: 26.273586364727485, MSE: 460.09019607843135, SSIM: 0.9466630985103914
1/1 [==============================] - 0s 37ms/step
Processed Image0339_016.png with PSNR: 23.96184787371007, MSE: 783.46, SSIM: 0.9356440929173401
1/1 [==============================] - 0s 35ms/step
Processed Image0322_042.png with PSNR: 30.337244093458963, MSE: 180.5, SSIM: 0.9713157616224201
1/1 [==============================] - 0s 40ms/step
Processed Image0347_002.png with PSNR: 28.25751629219587, MSE: 291.37345679012344, SSIM: 0.950510387147423
1/1 [=======